https://dacon.io/competitions/official/236229/overview/description

Dataset Info.

train.csv [파일] 
데이터 전처리 과정이 끝난 학습 데이터 (추가 데이터 전처리 과정을 진행하지 않습니다.)
RF 모델 하이퍼파라미터를 제출 시, 해당 데이터로 자동적으로 학습됩니다.
person_id: 유저별 고유 아이디
Sex: 성별
past_login_total: 과거(5월 8일 이전)에 로그인한 총 횟수
past_1_month_login: 과거 1달간 로그인한 총 횟수
past_1_week_login: 과거 1주간 로그인한 총 횟수
sub_size: 과거에 데이콘 대회에서의 총 제출 수
email_type: 가입한 이메일 종류
phone_rat: 폰으로 접속한 비율
apple_rat: 애플 기기로 접속한 비율
login: 로그인 여부 


sample_submission.csv [파일] - 제출 양식
RF 모델을 학습시킬 모델 하이퍼파라미터 목록
n_estimators:
기본값: 10
범위: 10 ~ 1000 사이의 양의 정수. 일반적으로 값이 클수록 모델 성능이 좋아지지만, 계산 비용과 시간도 증가합니다.
criterion:
기본값: 'gini'
옵션: 'gini', 'entropy'. 'gini'는 지니 불순도를, 'entropy'는 정보 이득을 기준으로 합니다.
max_depth:
기본값: None
범위: None 또는 양의 정수. None으로 설정하면 노드가 모든 리프가 순수해질 때까지 확장됩니다. 양의 정수를 설정하면 트리의 최대 깊이를 제한합니다.
min_samples_split:
기본값: 2
범위: 2 이상의 정수 또는 0과 1 사이의 실수 (비율을 나타냄, (0, 1] ). 내부 노드를 분할하기 위해 필요한 최소 샘플 수를 지정합니다.
min_samples_leaf:
기본값: 1
범위: 1 이상의 정수 또는 0과 0.5 사이의 실수 (비율을 나타냄, (0, 0.5] ). 리프 노드가 가져야 하는 최소 샘플 수를 지정합니다.
min_weight_fraction_leaf:
기본값: 0.0
범위: 0.0에서 0.5 사이의 실수. 리프 노드에 있어야 하는 샘플의 최소 가중치 비율을 지정합니다.
max_features:
기본값: 'auto'
옵션: 'auto', 'sqrt', 'log2', None 또는 양의 정수/실수. 최적의 분할을 찾기 위해 고려할 특성의 수 또는 비율을 지정합니다. 'auto'는 모든 특성을 사용함을 의미하며, 'sqrt'와 'log2'는 각각 특성의 제곱근과 로그2를 사용합니다. None은 'auto'와 동일하게 모든 특성을 의미합니다.
max_leaf_nodes:
기본값: None
범위: None 또는 양의 정수. 리프 노드의 최대 수를 제한합니다. None은 무제한을 의미합니다.
min_impurity_decrease:
기본값: 0.0
범위: 0.0 이상의 실수. 노드를 분할할 때 감소해야 하는 불순도의 최소량을 지정합니다.
bootstrap:
기본값: True
옵션: True, False. True는 부트스트랩 샘플을 사용하여 개별 트리를 학습시킵니다. False는 전체 데이터셋을 사용하여 각 트리를 학습시킵니다.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

In [3]:
df = pd.read_csv('./rf/train.csv')
test = pd.read_csv('rf/sample_submission.csv')

In [4]:
df

,person_id,Sex,past_login_total,past_1_month_login,past_1_week_login,sub_size,email_type,phone_rat,apple_rat,login
0,0,0,3.0,1.0,1.0,0.0,0,0.000000,0.000000,1
1,1,1,111.0,26.0,7.0,2.0,0,0.072072,0.000000,1
2,3,1,13.0,13.0,11.0,7.0,0,0.076923,1.000000,1
3,4,1,28.0,12.0,5.0,0.0,0,0.071429,0.071429,1
4,5,1,4.0,4.0,4.0,0.0,2,0.000000,0.000000,1
...,...,...,...,...,...,...,...,...,...,...
1304,2172,1,1.0,0.0,0.0,0.0,2,0.000000,0.000000,0
1305,2175,1,1.0,0.0,0.0,0.0,2,0.000000,0.000000,0
1306,2177,1,1.0,0.0,0.0,0.0,0,0.000000,1.000000,0
1307,2179,1,1.0,0.0,0.0,0.0,0,0.000000,1.000000,0


In [5]:
from sklearn.model_selection import LeaveOneOut,train_test_split , GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, roc_auc_score

In [6]:
# 성능확인 코드
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score, average_precision_score, fbeta_score
import numpy as np

import warnings

def get_clf_eval(y_test, pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    roc_score = roc_auc_score(y_test, pred)
    pr_score = average_precision_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    f2 = fbeta_score(y_test, pred, beta=2)
    # G-mean 계산
    tn, fp, fn, tp = confusion_matrix(y_test, pred).ravel()
    tpr = tp / (tp + fn)  # True Positive Rate
    tnr = tn / (tn + fp)  # True Negative Rate
    gmean = np.sqrt(tpr * tnr)

    print('오차행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도 : {1:.4f}, 재현율:{2:.4f},F1 스코어:{3:.4f}, f2 :{4:.4f}'.format(accuracy, precision, recall, f1, f2))
    print('ROC 스코어: {0:.4f}, PR 스코어 : {1:.4f}, G-mean : {2:.4f}'.format(roc_score, pr_score, gmean))

# 임계값에 따른 오차행렬및 스코어 -------->#임계값 최적 : 재현율기준(0.1)/f1기준(0.3)
from sklearn.preprocessing import Binarizer
thresholds = [0.1,0.15,0.2,0.25,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

def get_eval_by_threshold(y_test, pred_proba_c1, thresholds):
    for custom_threshold in thresholds:
        binarizer = Binarizer(threshold=custom_threshold).fit(pred_proba_c1)
        custom_predict = binarizer.transform(pred_proba_c1)
        # ROC 커브 계산
        fpr, tpr, thresholds = roc_curve(y_test, pred_proba_c1)
        roc_auc = auc(fpr, tpr)
        print("임곗값:", custom_threshold)
        print("ROC_AUC:", roc_auc)
        get_clf_eval(y_test, custom_predict)
        print("---------------------------------------------------------")

from sklearn.metrics import roc_curve, auc
def roc(model,name) :
    # 테스트 데이터에 대한 예측 확률 계산
    pred = model.predict_proba(X_test)[:, 1]

    # ROC 커브 계산
    fpr, tpr, thresholds = roc_curve(y_test, pred)

    # AUC 계산
    roc_auc = auc(fpr, tpr)

    # ROC 커브에 AUC 면적에 색을 입히는 코드 수정
    plt.figure(figsize=(10, 6))

    # ROC 커브 시각화 및 AUC 면적 색칠
    plt.fill_between(fpr, tpr, color='palegoldenrod', alpha=0.4, label=f'Area under curve (AUC = {roc_auc:.2f})')
    plt.plot(fpr, tpr, color='peru', lw=2)

    # 기본 설정
    plt.plot([0, 1], [0, 1], color='darkorange', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'{name}')
    plt.legend(loc="lower right")
    return plt.show()

In [7]:
df.columns

Index(['person_id', 'Sex', 'past_login_total', 'past_1_month_login',
       'past_1_week_login', 'sub_size', 'email_type', 'phone_rat', 'apple_rat',
       'login'],
      dtype='object')

In [8]:
X = df[['person_id', 'Sex', 'past_login_total', 'past_1_month_login',
       'past_1_week_login', 'sub_size', 'email_type', 'phone_rat', 'apple_rat',
       ]]
y = df[['login']]

In [9]:
#train/test 분리 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=17)


n_estimators:
기본값: 10
범위: 10 ~ 1000 사이의 양의 정수. 일반적으로 값이 클수록 모델 성능이 좋아지지만, 계산 비용과 시간도 증가합니다.
criterion:
기본값: 'gini'
옵션: 'gini', 'entropy'. 'gini'는 지니 불순도를, 'entropy'는 정보 이득을 기준으로 합니다.
max_depth:
기본값: None
범위: None 또는 양의 정수. None으로 설정하면 노드가 모든 리프가 순수해질 때까지 확장됩니다. 양의 정수를 설정하면 트리의 최대 깊이를 제한합니다.
min_samples_split:
기본값: 2
범위: 2 이상의 정수 또는 0과 1 사이의 실수 (비율을 나타냄, (0, 1] ). 내부 노드를 분할하기 위해 필요한 최소 샘플 수를 지정합니다.
min_samples_leaf:
기본값: 1
범위: 1 이상의 정수 또는 0과 0.5 사이의 실수 (비율을 나타냄, (0, 0.5] ). 리프 노드가 가져야 하는 최소 샘플 수를 지정합니다.
min_weight_fraction_leaf:
기본값: 0.0
범위: 0.0에서 0.5 사이의 실수. 리프 노드에 있어야 하는 샘플의 최소 가중치 비율을 지정합니다.
max_features:
기본값: 'auto'
옵션: 'auto', 'sqrt', 'log2', None 또는 양의 정수/실수. 최적의 분할을 찾기 위해 고려할 특성의 수 또는 비율을 지정합니다. 'auto'는 모든 특성을 사용함을 의미하며, 'sqrt'와 'log2'는 각각 특성의 제곱근과 로그2를 사용합니다. None은 'auto'와 동일하게 모든 특성을 의미합니다.
max_leaf_nodes:
기본값: None
범위: None 또는 양의 정수. 리프 노드의 최대 수를 제한합니다. None은 무제한을 의미합니다.
min_impurity_decrease:
기본값: 0.0
범위: 0.0 이상의 실수. 노드를 분할할 때 감소해야 하는 불순도의 최소량을 지정합니다.
bootstrap:
기본값: True
옵션: True, False. True는 부트스트랩 샘플을 사용하여 개별 트리를 학습시킵니다. False는 전체 데이터셋을 사용하여 각 트리를 학습시킵니다.

In [10]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=42, 
                            n_estimators=1000,
                            criterion='entropy',
                            max_depth=5, 
                            min_samples_split=3,
                            min_samples_leaf=3,
                            min_weight_fraction_leaf = 0,
                            max_features = None,
                            max_leaf_nodes = None,
                            min_impurity_decrease = 0,
                            bootstrap = True)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)
rf_pred_proba = rf.predict_proba(X_test)

get_clf_eval(y_test, rf_pred)
get_eval_by_threshold(y_test , rf_pred_proba[:,1].reshape(-1,1), thresholds)

c:\Users\hangil\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


오차행렬
[[227   3]
 [ 20  12]]
정확도: 0.9122, 정밀도 : 0.8000, 재현율:0.3750,F1 스코어:0.5106, f2 :0.4196
ROC 스코어: 0.6810, PR 스코어 : 0.3763, G-mean : 0.6084
임곗값: 0.1
ROC_AUC: 0.8547554347826086
오차행렬
[[186  44]
 [ 10  22]]
정확도: 0.7939, 정밀도 : 0.3333, 재현율:0.6875,F1 스코어:0.4490, f2 :0.5670
ROC 스코어: 0.7481, PR 스코어 : 0.2673, G-mean : 0.7456
---------------------------------------------------------
임곗값: 0.15
ROC_AUC: 0.8547554347826086
오차행렬
[[198  32]
 [ 13  19]]
정확도: 0.8282, 정밀도 : 0.3725, 재현율:0.5938,F1 스코어:0.4578, f2 :0.5307
ROC 스코어: 0.7273, PR 스코어 : 0.2708, G-mean : 0.7149
---------------------------------------------------------
임곗값: 0.2
ROC_AUC: 0.8547554347826086
오차행렬
[[206  24]
 [ 15  17]]
정확도: 0.8511, 정밀도 : 0.4146, 재현율:0.5312,F1 스코어:0.4658, f2 :0.5030
ROC 스코어: 0.7135, PR 스코어 : 0.2775, G-mean : 0.6898
---------------------------------------------------------
임곗값: 0.25
ROC_AUC: 0.8547554347826086
오차행렬
[[213  17]
 [ 16  16]]
정확도: 0.8740, 정밀도 : 0.4848, 재현율:0.5000,F1 스코어:0.4923, f2 :0.4969
ROC 스코어: 0.7130,

random_state=17, 
                            n_estimators=1050,
                            criterion='entropy',
                            max_depth=4, 
                            min_samples_split=3,
                            min_samples_leaf=4,
                            min_weight_fraction_leaf = 0,
                            max_features = None,
                            max_leaf_nodes = None,
                            min_impurity_decrease = 0,
                            bootstrap = True
0.8296949944656254

# 1차 3/18
{'bootstrap': True,
 'max_depth': 4,
 'min_samples_leaf': 2,
 'min_samples_split': 3,
 'n_estimators': 1000}
 
 # 2차 3/20
 1000,entropy,None,3,2,0,None,None,0,TRUE

In [11]:
from sklearn.model_selection import LeaveOneOut,train_test_split , GridSearchCV, StratifiedKFold
from sklearn.model_selection import GridSearchCV

rf_grid = RandomForestClassifier(random_state=42)
#cv = StratifiedKFold(n_splits=5)
cv = LeaveOneOut()
# 파라미터를 딕셔너리로 저장
param_grid = {'n_estimators' : [1000,900,800],
            'criterion' : ['gini', 'entropy'],
            'max_depth' : [5,4,3],
            'min_samples_leaf' : [1,2,3],
            }
print(param_grid)
grid_search = GridSearchCV(rf_grid, param_grid=param_grid, cv=cv, return_train_score=True, refit=True)

grid_search.fit(X_train, y_train)
score = grid_search.score(X_test, y_test)
print("검증 세트 점수: {:.2f}".format(score))
print(grid_search.best_estimator_)

{'n_estimators': [1000, 900, 800], 'criterion': ['gini', 'entropy'], 'max_depth': [5, 4, 3], 'min_samples_leaf': [1, 2, 3]}


c:\Users\hangil\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\hangil\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\hangil\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\hangil\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:1152: DataConvers

KeyboardInterrupt: 

In [ ]:
final_score = grid_search.score(X_test, y_test)
print("테스트 세트 점수: {:.2f}".format(final_score))

테스트 세트 점수: 0.90


2차